In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import nltk

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel

# spacy for lemmatization
import spacy


In [2]:
#!pip install -r requirements.txt

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import nltk

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

stop_words = nltk.corpus.stopwords.words('dutch')
stop_words.extend(['kb', 'pdf','nationaal','rapporteur'])

#Install Dutch Spacy with python -m spacy download nl_core_news_sm
#!python -m spacy download nl_core_news_sm
import nl_core_news_sm

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#python3 -m spacy download en
nlp = nl_core_news_sm.load()

# Execute Applications/python 3.7/Install Certificates.command
import nltk
nltk.download('stopwords')

df = pd.read_excel('pdf_pages.xlsx')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\oprokopenk001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def clean_data(data):
    # Put string in list
    if type(data) == type('string'):
        data = [data]
    #for sent in data[0:100]:
        #print(sent.dtype)
    
    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

    # Remove special charcters
    pattern = r'[^a-zA-z0-9\s]' if not True else r'[^a-zA-z\s]'
    data = [re.sub(pattern, '', sent) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    data = [re.sub("\_", "", sent) for sent in data]
    return data


def parse_data(data):

    data = clean_data(data)

    def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

    data_words = list(sent_to_words(data))

    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)  # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
        # Define functions for stopwords, bigrams, trigrams and lemmatization
    def remove_stopwords(texts):
        return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

    def make_bigrams(texts):
        return [bigram_mod[doc] for doc in texts]

    def make_trigrams(texts):
        return [trigram_mod[bigram_mod[doc]] for doc in texts]

    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent))
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return texts_out
    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)
    
    print(data_words_bigrams)

    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    return data_lemmatized


## Prepare data for model

In [ ]:
# Convert to list
data = df.loc[0:10000].par_id.values.tolist()
data_lemmatized = parse_data(data)

# Do lemmatization keeping only noun, adj, vb, adv

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]



In [ ]:
# Do lemmatization keeping only noun, adj, vb, adv

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [ ]:
len(corpus)

## LSI

In [ ]:
# Try methods: tf-idf, LSI and LDA
lsi = gensim.models.LsiModel(corpus, id2word=id2word, num_topics=13)

doc = "mensenhandel loverboy"
vec_bow = id2word.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
print(vec_lsi)

index = gensim.similarities.MatrixSimilarity(lsi[corpus])
sims = index[vec_lsi]
print(list(enumerate(sims)))

## TF-IDF

In [ ]:
tfid = TfidfModel(corpus)
doc = data_lemmatized
total= pd.DataFrame([], columns=['f','s','t'])


In [ ]:
for i in range(9598, len(doc)):
    vec_bow = id2word.doc2bow(doc[i])
    vector = tfid[vec_bow]
    d = pd.DataFrame([], columns=['f','s','t'])
    for val in vector:
        word = id2word.get(val[0])
        d=d.append(pd.DataFrame([[i, word,  val[1]]], columns=['f','s','t']))
    total=total.append(d)

In [ ]:
i

In [ ]:
total.head()

In [ ]:
total.to_csv('all_keywords2.csv')

In [31]:
total1=pd.read_csv('all_keywords2.csv')
total2=pd.read_csv('all_keywords.csv')
total=total1.append(total2)

In [32]:
total.shape

(1079347, 4)

In [45]:
#keywords = total.groupby('f')['t'].nlargest(100, 't')
#keywords
keywords = total.groupby('f').apply(lambda x: x.nlargest(100,'t'))


In [ ]:
keywords

In [ ]:
total.head()

## Search

In [5]:
    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent))
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return texts_out

In [108]:
def search(doc1):
    total1=pd.read_csv('all_keywords2.csv')
    total2=pd.read_csv('all_keywords.csv')
    total=total1.append(total2)
    keywords = total.groupby('f').apply(lambda x: x.nlargest(100,'t'))
    doc1.split()
    allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
    texts_out = []
    for sent in doc1.split():
        doc = nlp("".join(sent))
        print(doc)
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    doc=texts_out
    training_data_x = pd.read_excel("synonyms.xlsx")
    X_train = training_data_x.as_matrix()
    syn=pd.DataFrame()
    for index, val in enumerate(doc):
        try:
            syn[str(index)]=(X_train[np.nonzero(X_train ==(val))[0][0],:])
        except:
            syn[str(index)]=" "
    syn_sp=syn.iloc[:,0]
    syn_sp=syn_sp[syn_sp.notnull()]
    search_results= pd.DataFrame([], columns=['f'])
    for i in range(0, len(doc)):
        syn_sp=syn.iloc[:,i]
        syn_sp=syn_sp[syn_sp.notnull()]
        for j in range(0,len(syn_sp)):
            search_results=search_results.append(pd.DataFrame(keywords.f[keywords.s==syn_sp[j]]))
    search_results=search_results.drop_duplicates() 
    return search_results


In [112]:
search_results=search('kinderen')

kinderen


C:\Users\oprokopenk001\AppData\Anaconda\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app


In [113]:
search_results

,f
"(1, 43)",1
"(4, 343)",4
"(5, 421)",5
"(7, 468)",7
"(8, 582)",8
"(9, 736)",9
"(17, 1468)",17
"(19, 1552)",19
"(20, 1660)",20
"(21, 1794)",21


In [6]:
doc1=("kinderen loverboy")

doc1.split()

['kinderen', 'loverboy']

In [7]:
allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
texts_out = []
for sent in doc1.split():
    doc = nlp("".join(sent))
    print(doc)
    texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])


kinderen
loverboy


In [8]:
doc=texts_out

In [9]:
doc[1][0]

'loverboy'

In [13]:
training_data_x = pd.read_excel("synonyms.xlsx")
X_train = training_data_x.as_matrix()

C:\Users\oprokopenk001\AppData\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [84]:
syn=pd.DataFrame()

In [97]:
for index, val in enumerate(doc):
    try:
        syn[str(index)]=(X_train[np.nonzero(X_train ==(val))[0][0],:])
    except:
        syn[str(index)]=" "


In [98]:
 syn


,0,1
0,koter,
1,loot,
2,kind,
3,spruit,
4,NaN,
5,NaN,
6,NaN,
7,NaN,
8,NaN,
9,NaN,


In [99]:
syn_sp=syn.iloc[:,0]

syn_sp=syn_sp[syn_sp.notnull()]

0     koter
1      loot
2      kind
3    spruit
Name: 0, dtype: object

In [106]:
search_results= pd.DataFrame([], columns=['f'])
for i in range(0, len(doc)):
    syn_sp=syn.iloc[:,i]
    syn_sp=syn_sp[syn_sp.notnull()]
    for j in range(0,len(syn_sp)):
        search_results=search_results.append(pd.DataFrame(keywords.f[keywords.s==syn_sp[j]]))
                                             

In [107]:
search_results=search_results.drop_duplicates() 
search_results.shape

(1171, 1)

In [103]:
doc[1][0]

'loverboy'

In [22]:
 len(doc)

2

In [ ]:
return_res=keywords.f[keywords.s==doc[1][0]]

In [ ]:
return_res.shape

In [ ]:
data_lemmatized[0]

In [ ]:
id2word.doc2bow("kinderen rapporteur".split())

In [ ]:
"kinderen rapporteur".split()

In [ ]:
doc = parse_data("kinderen loverboy")
vec_bow = id2word.doc2bow(doc[0])
vec_bow


In [ ]:
doc=pd.DataFrame(np.array([["kinderen loverboy"], ["mensenhandel loverboy"]]))

In [ ]:
doc

In [ ]:
X_train

In [ ]:
try:
    print(X_train[np.nonzero(X_train =='misbruik')[0][0],:])
except:
    print ("no synonyms")



In [ ]:
from nltk.corpus import wordnet
syns = wordnet.synsets("dog")
print(syns)

In [ ]:
from nltk.corpus import wordnet
synonyms = []
antonyms = []

for syn in wordnet.synsets("active"):
    for l in syn.lemmas():
    synonyms.append(l.name())
    if l.antonyms():
    antonyms.append(l.antonyms()[0].name())

    print(set(synonyms))
    print(set(antonyms))

In [ ]:
for i in range(0,1000):
    #if type(i)!=type(string)):
    try:
        cl_string = re.sub('\S*@\S*\s?', '', data[i])
        print(cl_string)
        data_cl.append(cl_string)
    except:
        print(i)